In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('output.csv',  index_col=[0])

In [3]:
df.head()

,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_name,lat,lng,population,density,Year
0,43,Good,Ozone,1,Flagstaff,Arizona,35.1872,-111.6194,79624.0,428.0,2022
1,42,Good,Ozone,2,Flagstaff,Arizona,35.1872,-111.6194,79624.0,428.0,2022
2,43,Good,Ozone,2,Flagstaff,Arizona,35.1872,-111.6194,79624.0,428.0,2022
3,42,Good,Ozone,2,Flagstaff,Arizona,35.1872,-111.6194,79624.0,428.0,2022
4,43,Good,Ozone,2,Flagstaff,Arizona,35.1872,-111.6194,79624.0,428.0,2022


Encoding categorical data: `Defining Parameter`, `city_ascii` and `state_name`.

In [4]:
df['Defining Parameter'] = df['Defining Parameter'].astype('category')
df['Defining Parameter'] = df['Defining Parameter'].cat.codes
df['city_ascii'] = df['city_ascii'].astype('category')
df['city_ascii'] = df['city_ascii'].cat.codes
df['state_name'] = df['state_name'].astype('category')
df['state_name'] = df['state_name'].cat.codes
df.head()

,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_name,lat,lng,population,density,Year
0,43,Good,2,1,25,1,35.1872,-111.6194,79624.0,428.0,2022
1,42,Good,2,2,25,1,35.1872,-111.6194,79624.0,428.0,2022
2,43,Good,2,2,25,1,35.1872,-111.6194,79624.0,428.0,2022
3,42,Good,2,2,25,1,35.1872,-111.6194,79624.0,428.0,2022
4,43,Good,2,2,25,1,35.1872,-111.6194,79624.0,428.0,2022


### Separar os dados para treino, teste e validação
Aplicaremos a função `train_test_split` duas vezes para obter os três conjuntos: treino, teste e validação.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
y = df.Category
X = df.drop('Category', axis=1)
train_ratio = 0.7
test_ratio = 0.20
validation_ratio = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_ratio, random_state=1)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=validation_ratio/(train_ratio+test_ratio))

In [7]:
# iniciando o MLFlow
import mlflow

In [8]:
mlflow.set_experiment('air-quality-category')

Traceback (most recent call last):
  File "C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 270, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 394, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 1049, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\store\tracking\file_store.py", line 1042, in _read_helper
    result = read_yaml(root, file_name)
  File "C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\utils\file_utils.py"

<Experiment: artifact_location='file:///C:/Users/Anna%20Luiza/Documents/DataScience-Projeto2/mlruns/1', experiment_id='1', lifecycle_stage='active', name='air-quality-category', tags={}>

### Treinamentos

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [9]:
mlflow.start_run(experiment_id='1', run_name='air-quality-category')

<ActiveRun: >

In [11]:
# create and fit knn
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [12]:
mlflow.sklearn.log_model(knn, 'knn')

ModelInfo(artifact_path='knn', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.10.1', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.1.2', 'serialization_format': 'cloudpickle', 'code': None}}, model_uri='runs:/94b9f72ddf4f4372977556de239de789/knn', model_uuid='91dc0eaf82344067ac438268e4d0c790', run_id='94b9f72ddf4f4372977556de239de789', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-09-12 16:59:13.932328', mlflow_version='1.28.0')

In [13]:
# create and fit naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [14]:
mlflow.sklearn.log_model(gnb, 'gnb')

ModelInfo(artifact_path='gnb', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.10.1', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.1.2', 'serialization_format': 'cloudpickle', 'code': None}}, model_uri='runs:/94b9f72ddf4f4372977556de239de789/gnb', model_uuid='9d536f18670a44d6acd746f1a428bd99', run_id='94b9f72ddf4f4372977556de239de789', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-09-12 16:59:49.332729', mlflow_version='1.28.0')

In [12]:
# create and fit MLP with PyTorch


In [13]:
mlflow.sklearn.log_model(mlp, 'mlp')

ModelInfo(artifact_path='mlp', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.10.1', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.1.2', 'serialization_format': 'cloudpickle', 'code': None}}, model_uri='runs:/7682d5206fe04e67b71520f35d8ab680/mlp', model_uuid='91391f6b6788406faea3e78a9d6256d0', run_id='7682d5206fe04e67b71520f35d8ab680', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-09-11 14:45:42.328790', mlflow_version='1.28.0')

In [15]:
# create and fit Random Forest
random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [16]:
mlflow.end_run()

### Seleção de hiper-parâmetros

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [17]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

#### KNN

In [17]:
params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [19]:
grid_search = GridSearchCV(estimator=tree, 
                           param_grid=params, 
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy")

In [20]:
%%time
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
CPU times: total: 19.6 s
Wall time: 4min 54s


GridSearchCV(cv=4, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 20],
                         'min_samples_leaf': [5, 10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

In [21]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)

In [23]:
tree_best = grid_search.best_estimator_

#### Gaussian NB

In [21]:
import numpy as np
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV

params_gnb = {'var_smoothing': np.logspace(0,-9, num=100)}
grid_search_gnb = GridSearchCV(estimator=gnb, 
                 param_grid=params_gnb, 
                 cv=4, 
                 verbose=1, 
                 scoring='accuracy') 
Data_transformed = PowerTransformer().fit_transform(X_test)
grid_search_gnb.fit(Data_transformed, y_test);

grid_search_gnb.best_estimator_


Fitting 4 folds for each of 100 candidates, totalling 400 fits


GaussianNB(var_smoothing=1e-05)

#### Multi-layer perceptron

In [14]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
distributions = {'hidden_layer_sizes': (5,2),
                 'activation': ['identity', 'logistic', 'tanh', 'relu'],
                 'solver': ['lbfgs', 'sgd', 'adam']}
random_search_mlp = RandomizedSearchCV(mlp, distributions, random_state=0)
random_search_mlp.fit(X_train, y_train)
random_search_best = random_search_mlp.best_estimator_

C:\Users\Anna Luiza\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


* Atenção: o scikit learn não foi indicado para uso com MLP, então vamos trocar pelo PyTorch

#### Random Forest

### Selecionar melhor modelo e aplicar otimização de acordo com os melhores hiper-parâmetros

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [31]:
def evaluate_model(dt_classifier):
    print("Test Accuracy :", accuracy_score(y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))

In [ ]:
# avaliar o melhor modelo de knn

In [42]:
evaluate_model(gnb_best)

Test Accuracy : 0.6441528864953127
Test Confusion Matrix:
[[172335      0   4262      0      0    929]
 [   108      0     39      0      0      0]
 [ 64495      0   7454      0      0   2256]
 [  5618      0   1153      0      0    199]
 [ 14541      0   2750      0      0    554]
 [  1598      0    874      0      0    101]]


In [ ]:
# avaliar o melhor modelo de MLP

In [ ]:
# avaliar o melhor modelo de Random Forest